Timestamp: 6/27/2024 4:04 PM

# *Web Scraping OPM Song Lyrics*

 **Important**

The activity uses a [website](https://www.azlyrics.com/) for the list of OPM songs and singers, which may not contain all OPM songs or artists. 

In [13]:
from bs4 import BeautifulSoup, Comment
import bs4
import pandas as pd
import requests
import re
import time

In [14]:
print("BeautifulSoup4: ", bs4.__version__)
print("Pandas: ", pd.__version__)
print("Requests: ", requests.__version__)
print("RegEx: ", re.__version__)

BeautifulSoup4:  4.12.3
Pandas:  2.2.2
Requests:  2.32.3
RegEx:  2.2.1


# Preliminaries

## **Dataset Description**

The song and singer data was extracted from [this](https://www.azlyrics.com/) website, which contains lyrics for each song of a specified artist.

To isolate the data to include only OPM songs, a list of Filipino singers was taken from [this](https://en.wikipedia.org/wiki/List_of_Filipino_singers) website.

In [24]:
#singers = ["Dong Abay", "Bayani Agbayani", "Joey Albert", "Barbie Almalbis", "Amapola", "Cynthia Alexander", "Freddie Aguilar", "Hajji Alejandro", "Rachel Alejandro", "Joey Ayala", "Kyline Alcantara", "Ogie Alcasid", "Dely Atay-Atayan", "Dingdong Avanzado", "Carol Banawa", "Edward Barber", "Bayang Barrios", "Carlos Balcells", "Bassilyo", "Heber Bartolome", "Jimmy Bondoc", "Jovit Baldivino", "Christian Bautista", "Beabadoobee", "Zild Benitez", "Mark Bautista", "Janine Berdin", "Blakdyak", "Rico Blanco", "Alisah Bonaobra", "Andrea Brillantes", "Ely Buendia", "Noel Cabangon", "Lito Camo", "Ryan Cayabyab", "Billy Crawford", "John Cadeliña", "Jose Mari Chan", "Kim Chiu", "Kean Cipriano", "Sam Concepcion", "Pilita Corrales", "Teddy Corpuz", "Roel Cortez", "Yeng Constantino", "Donna Cruz", "Radha Cuadrado", "Sharon Cuneta", "Anne Curtis", "Claire de la Fuente", "Glaiza de Castro", "Dianne dela Fuente", "Dingdong Dantes", "Karina Constantino David", "JM de Guzman", "Klarisse de Guzman", "Mrld", "Moira Dela Torre", "Joey de Leon", "Kisses Delavin", "Enchong Dee", "Lourd de Veyra", "Francine Diaz", "Elaine Duran", "Jason Dy", "Andrew E.", "Kyle Echarri", "Maymay Entrata", "Bituin Escalante", "Darren Espanto", "Brenan Espartinez", "Karla Estrada", "Heart Evangelista", "Seth Fedelin", "Pops Fernandez", "Juris Fernandez", "Florante", "Rose Fostanes", "Vice Ganda", "Maricris Garcia", "Tootsie Guevara", "Janno Gibbs", "Enrique Gil", "Nikki Gil", "Rachelle Ann Go", "Sarah Geronimo", "King Girado", "Alex Gonzaga", "Toni Gonzaga", "Gary Granada", "Catriona Gray", "Mike Hanopol", "Solenn Heussaff", "Agot Isidro", "Ramon Jacinto", "Jaya", "RJ Jimenez", "Jing Hidalgo Santos", "Karylle", "Angela Ken", "Kyla", "Yasmien Kurdi", "Juan Karlos Labajo", "Raymond Lauchengco", "Kris Lawrence", "Kuh Ledesma", "Celeste Legaspi", "Ronnie Liang", "Xian Lim", "Nadine Lustre", "Chito Miranda", "Jed Madela", "Elmo Magalona", "Francis Magalona", "Bamboo Manalac", "Jennylyn Mercado", "Gian Magdangal", "Jolina Magdangal", "Michael Pangilinan", "Maine Mendoza", "Sam Milby", "Lani Misalucha", "Vina Morales", "Morissette", "Kitchie Nadal", "Roselle Nava", "Sitti Navarro", "Vhong Navarro", "Arthur Nery", "Martin Nievera", "Nina", "Grace Nono", "Paolo Onesa", "Daryl Ong", "Daniel Padilla", "Kylie Padilla", "Zsa Zsa Padilla", "Manny Pacquiao", "Inigo Pascual", "Piolo Pascual", "Jett Pangan", "Imelda Papin", "Eddie Peregrina", "Arnel Pineda", "Lovi Poe", "Pokwang", "Marcelito Pomoy", "Richard Poon", "Chickoy Pura", "Yassi Pressman", "Rico J. Puno", "Myk Perez", "Odette Quesada", "Angeline Quinto", "Zia Quizon", "Jay R", "Khalil Ramos", "John Raymundo", "April Boy Regino", "Sheryn Regis", "James Reid", "Willie Revillame", "Manilyn Reynes", "Alden Richards", "Ariel Rivera", "Jamie Rivera", "Marian Rivera", "Olivia Rodrigo", "Wilbert Ross", "Karl Roy", "Myra Ruaro", "Sarah Geronimo", "Lea Salonga", "Unique", "Janella Salvador", "Maja Salvador", "Sampaguita", "Julie Anne San Jose", "Josh Santana", "Randy Santiago", "Aicelle Santos", "Erik Santos", "Gerald Santos", "Judy Ann Santos", "Vehnee Saturno", "Aiza Seguerra", "Vic Sotto", "Marco Sison", "Jay-R Siaboc", "Pepe Smith", "Max Surban", "KZ Tandingan", "Daniel Tagibao", "Ivy Violan", "Michael V", "Regine Velasquez", "Nikki Valdez", "Gary Valenciano", "Rey Valera", "Viktoria", "Jessica Villarubin", "Yoyoy Villame", "Cris Villonco", "Jona Viray", "Katrina Velarde", "Victor Wood", "Richard Yap", "Mitoy Yonting", "Jake Zyrus", "Jessa Zaragoza"]
singers = ["Jake Zyrus"]

singer_urls = []

for singer in singers:
    # Transforms the singer's names into codes that can be found in the website
    singer_urls.append(singer.lower().replace(" ", "")) 

print(singer_urls)

['jakezyrus']


## **Variables**

Here are the list of variables that would be used

In [16]:
singers_that_have_webpages = []
webpages_that_exist = []
all_song_links = []
all_song_titles = []
all_song_lyrics = []

# Data Extraction

## **Webpage Checking**

First, we check if a webpage exists for each singer.

If a page does not exist for a singer, we discard the singer.

In [26]:
for singer_url_number in range(len(singers)):

    # Gets the webpage with the singer codes
    response = requests.get('https://www.azlyrics.com/' + singer_urls[singer_url_number][0] + '/' + singer_urls[singer_url_number] + '.html')
    
    # Removes singers that do not have webpages
    if response.status_code == 200:
        webpages_that_exist.append(response)
        singers_that_have_webpages.append(singers[singer_url_number])
        
    time.sleep(0.2)

    print(response)
    
print("Singers with webpages:", singers_that_have_webpages)


<Response [403]>
Singers with webpages: []


## **Songs URL Extraction**

Each singer's webpage was opened and checked for "a" tags.

Tags that contained the *href* attribute with a value that had a prefix of "/lyrics/" were then isolated to get the url for the song lyrics.

In [18]:
for webpage_count in range(len(webpages_that_exist)):
    
    # Finds all <a> tags in the singer webpages 
    all_a_tags = BeautifulSoup(webpages_that_exist[webpage_count].text, 'html.parser').find_all('a', target="_blank")

    # Filters the <a> tags to get only the ones with the "/lyrics/" prefix 
    filtered_a_tags = [a for a in all_a_tags if a['href'].startswith("/lyrics/")]

    # Gets the links for all the song lyrics in the singer webpage
    song_link = [a['href'] for a in filtered_a_tags]
    all_song_links.append(song_link)

print("Links for each song:", all_song_links)

Links for each song: []


## **Song Title and Lyrics Extraction**

**Each song's titles and lyrics were extracted in 6 steps**


1. The url for the webpage containing each song's lyrics was opened and converted into text form.

2. For the song's title, all "h1" tags were extracted from the webpage
3. A RegEx function was used to isolate the text inside quotation marks in the tag to get the title
4. For the song lyrics, all "div" tags were extracted from the webpage
5. The "div" tags were than checked if they contain comments
6. Those that do were checked again if they have comments directly inside it with the text *"Sorry about that"* to get the lyrics

In [19]:
for singer_number in range(len(singers_that_have_webpages)):

    song_title = []
    song_lyrics = []

    for song in all_song_links[singer_number]:
        
        # Gets the song lyrics webpage
        response = requests.get('https://www.azlyrics.com' + song)
        
        # Gets all the <div> tags in the webpage
        all_div_tags = BeautifulSoup(response.text, 'html.parser').find_all('div')

        # Finds all the h1 tags containing song titles
        h1_tags = BeautifulSoup(response.text, 'html.parser').find_all('h1')

        for tag in h1_tags:
            
            # Isolates the song title and adds the song titles to a list 
            pattern = r'["\'](.*?)["\']'
            song_title.append(re.search(pattern, tag.text).group(1)) 
               
        for div in all_div_tags:

            # Gets all <div> tags with comments inside them
            all_comments = div.find_all(string=lambda text: isinstance(text, Comment))
            
            time.sleep(0.2)
            
            for child in div.children:

                # Filters the <div> tags to filter only the ones with the comment "Sorry about that" in it
                if isinstance(child, Comment) and "Sorry about that" in child:
                    song_lyrics.append(div)
                    break
        
        time.sleep(0.2)

    all_song_titles.append(song_title)
    all_song_lyrics.append(song_lyrics)

In [20]:
for count in range(len(singers_that_have_webpages)):

    print(singers_that_have_webpages[count] + ":", all_song_titles[count])

## **Storing the Data**

The extracted song titles and lyrics were then stored in a .csv file, with each singer being the variable 

In [27]:
# Creates a .csv file
f = open("S:\\PSHS\\Grade 10\\SIP\\De La Salle\\Projects\\Web Scraping Activity\\all_opm_songs.csv", "w", encoding="utf-8")

for singer_number in range(len(singers_that_have_webpages)):
    
    f.write(singer)

    if (singer_number < len(singers_that_have_webpages) - 1):
        
        f.write(",")

f.write('\n')

# Outer loop for every song
for song_number in range(len(max(all_song_lyrics, key=len))):
    
    # Inner loop for every singer
    for singer_number in range(len(singers_that_have_webpages)):

        try:

            # Puts the lyrics into the .csv file
            song_lyrics = all_song_titles[singer_number][song_number].replace('\n', ' ') + ": " + all_song_lyrics[singer_number][song_number].text.lstrip()
            f.write(song_lyrics.replace(',', ' ').replace('\n', ' / '))

            if (singer_number < len(singers_that_have_webpages) - 1):

                f.write(",")
        
        except IndexError:
            continue

    f.write('\n')

f.close()

pd.read_csv("all_opm_songs.csv")


ValueError: max() iterable argument is empty

# *Exploratory Data Analysis*

A few questions were made which can be answered by Exploratory Data Analysis

1. Which singer has the most songs? Which had the lowest?
2. What word was the most frequently used out of all words in all song lyrics? What word was the least frequent?

In [28]:
hfi_dataset = pd.read_csv('All OPM Songs.csv')
hfi_dataset.head()

,Jake Zyrus,Jake Zyrus.1
0,Espasyo: Kwento-kwenta Kwento-kwenta / Kwento...,Hiling: Nag-iisang pag-ibig ang nais makamit y...
1,Akrostik: D - Di ako si Bamboo Chito Parokya ...,Bagyo: Naririnig kay lakas ng pintig / Nagbaba...
2,Aba-Aba: Kasalanan ko kung bakit ka nagkaganya...,I: When you wake up each mornin' / And you fee...
3,Perpekto: Ikaw ay nagdaramdam / Puso ay nagdur...,DNM: 'Di mo na kailangan / Mag-abala mag-alal...
4,Kukote: May araw na o laging gabi / Sa mata ng...,Diamond: I have to admit I am in the rough / T...


## **EDA Question 1**

Question: Which singer has the most songs? Which had the lowest?

In order to compare the **frequency** of each word, we will split each song's lyrics into words.

Then, we create a bar graph for each word and their frequency.

In [29]:
for lyrics in all_song_lyrics:

    words = lyrics.replace(',', ' ').split(' ')

    print(words)

In [30]:
hfi_dataset[]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Jake Zyrus    14 non-null     object
 1   Jake Zyrus.1  11 non-null     object
dtypes: object(2)
memory usage: 356.0+ bytes
